# Set-up

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [4]:
#!pip install pulp

In [5]:
# install pulp
from pulp import *

In [6]:
# check current path
import os
print(os.getcwd())
'''
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive/')

path_gdrive = '/content/drive/Shareddrives/Enova/2023'
os.chdir(path_gdrive)
print(os.getcwd())
'''

path_gdrive = '/Users/anthony/Desktop/Enova/data'
os.chdir(path_gdrive)
print(os.getcwd())

/Users/anthony/Desktop/Enova/codes
/Users/anthony/Desktop/Enova/data


# Load Data

In [7]:
## Set up the data and random seed

# get validation
validation = pd.read_csv('optimization.csv') 
validation.head()

,promotion_type_suggest,ltv,conv_ind,probs_bronze,probs_silver,probs_gold
0,gold,1865.039580,1,0.102068,0.572957,2.174304
1,bronze,789.221262,1,0.986791,5.539325,21.021076
2,NaN,0.000000,0,0.029707,0.166758,0.632826
3,silver,2674.527593,1,0.573009,3.216572,12.206504
4,silver,332.631585,1,0.181862,1.020876,3.874101


In [8]:
validation.shape

(10000, 6)

Since our work is almost 90% done. The rest of work is to:
- calculate profit: ltv - promotion cost
- make decision on promotion approval -- because sometimes the ltv is lower than the promotion cost

In [9]:
# get data
df = validation.copy()

# calculate profit
mapping = {'gold': 1400, 'silver': 1100, 'bronze': 500, 'none': 0}
df['promotion_type_suggest'] = df['promotion_type_suggest'].fillna('none')
df['cost'] = df['promotion_type_suggest'].map(mapping)
df['profit'] = df['ltv'] - df['cost']

# approve those with profit > 0
df['approval'] = df['profit'] > 0.0

## Final output, if no budget given

In [10]:
df_output = df.copy()

# remove 'none'
df_output['promotion_type_suggest'] = df['promotion_type_suggest'].replace('none', np.nan)

# preview
#df.head(5)

total_profit = df[df['approval'] == True]['profit'].sum()
print(f'Total profit generated from promotion: {total_profit}')

Total profit generated from promotion: 5728072.369832329


In [11]:
# adjust
df_output['approval'] = df_output['approval'].apply(lambda x: None if x == False else x)
df_output['promotion_type'] = df_output['promotion_type_suggest'] * df_output['approval']

# drop columns
df_output = df_output.drop(columns=['cost', 'approval', 'probs_bronze', 'probs_gold', 'probs_silver', 'promotion_type_suggest'])

In [12]:
df_output

,ltv,conv_ind,profit,promotion_type
0,1865.039580,1,465.039580,gold
1,789.221262,1,289.221262,bronze
2,0.000000,0,0.000000,NaN
3,2674.527593,1,1574.527593,silver
4,332.631585,1,-767.368415,NaN
...,...,...,...,...
9995,1146.205525,1,46.205525,silver
9996,0.000000,0,0.000000,NaN
9997,2538.689969,1,1438.689969,silver
9998,2861.996002,1,1761.996002,silver


In [13]:
# sanity check
total_profit = df_output[df_output['promotion_type'].notna()]['profit'].sum()
print(f'Total profit generated from promotion: {total_profit}')

# export to csv
df_output.to_csv('submission_1_no_constraint.csv', index=False)

Total profit generated from promotion: 5728072.369832329


## Optimization

In real-world, the company might not have unlimited budget for promotions. That is why we need a budget constrain and do the optimization within.

Recall that the `promotion_type_suggest` is made by finding the minimal possible values of conversion probabilities that is higher than our threshold probability. The goal is to find the least possible promotion while making sure the users will still have high chance (in our case is > 90%) to subscribe. So, we won't worry about the MaxMin complexity among different promotion suggestions. Instead, we can simply focus on maximing profit while having a upper bond for total promotion cost

In [14]:
# get data
df = validation.copy()

# calculate profit
mapping = {'gold': 1400, 'silver': 1100, 'bronze': 500, 'none': 0}
df['promotion_type_suggest'] = df['promotion_type_suggest'].fillna('none')
df['cost'] = df['promotion_type_suggest'].map(mapping)
df['profit'] = df['ltv'] - df['cost']

# set target column
df['approval'] = None

In [15]:
df.head()

,promotion_type_suggest,ltv,conv_ind,probs_bronze,probs_silver,probs_gold,cost,profit,approval
0,gold,1865.039580,1,0.102068,0.572957,2.174304,1400,465.039580,None
1,bronze,789.221262,1,0.986791,5.539325,21.021076,500,289.221262,None
2,none,0.000000,0,0.029707,0.166758,0.632826,0,0.000000,None
3,silver,2674.527593,1,0.573009,3.216572,12.206504,1100,1574.527593,None
4,silver,332.631585,1,0.181862,1.020876,3.874101,1100,-767.368415,None


In [16]:
# Define the data for the problem as a DataFrame
prob = LpProblem("Promotion decision problem", sense=LpMaximize)

# Define the data and budget
df_lp = df.copy() # save a copy of df
df_lp = df_lp[['cost', 'profit']]
budget = 500000

# Define the decision variables
x = LpVariable.dicts("user", df_lp.index, lowBound=0, cat='Binary')

# Define the objective function
prob += sum([df_lp.loc[p, 'profit'] * x[p] for p in df_lp.index])

# Define the constraints
prob += sum([df_lp.loc[p, 'cost'] * x[p] for p in df_lp.index]) <= budget

# Solve the problem
prob.solve()


# Print the results
print("Status: ", prob.status)
print("Optimal Solution:")
for p in df_lp.index:
    print("{}: {}".format(p, x[p].varValue))
print("Total Profit: {}".format(prob.objective.value()))

/opt/anaconda3/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/sj/sfzzpb692rx3g93bqf9qgygh0000gn/T/1e08f37922a548e1a914f66c9a481114-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/sj/sfzzpb692rx3g93bqf9qgygh0000gn/T/1e08f37922a548e1a914f66c9a481114-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 35655 RHS
At line 35657 BOUNDS
At line 44570 ENDATA
Problem MODEL has 1 rows, 8912 columns and 8912 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1.88868e+06 - 0.01 seconds
Cgl0004I processed model has 1 rows, 5788 columns (5788 integer (5787 of which binary)) and 5788 elements
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.2
Cbc0038I Solution found of -1.88842e+06
Cbc0038I Cleaned solution of -1.88842e+06
Cbc0038I Be

In [17]:
approval = []
for p in df_lp.index:
    decision = x[p].varValue
    if x[p].varValue == None:
        decision = 0.0
        approval.append(decision)
    else:
        approval.append(decision)

# check if the opmization is doing right
pd.Series(approval).value_counts()

0.0    9006
1.0     994
dtype: int64

In [18]:
df['approval'] = pd.Series(approval)
df.head(5)

,promotion_type_suggest,ltv,conv_ind,probs_bronze,probs_silver,probs_gold,cost,profit,approval
0,gold,1865.039580,1,0.102068,0.572957,2.174304,1400,465.039580,0.0
1,bronze,789.221262,1,0.986791,5.539325,21.021076,500,289.221262,0.0
2,none,0.000000,0,0.029707,0.166758,0.632826,0,0.000000,0.0
3,silver,2674.527593,1,0.573009,3.216572,12.206504,1100,1574.527593,0.0
4,silver,332.631585,1,0.181862,1.020876,3.874101,1100,-767.368415,0.0


In [19]:
df_output2 = df.copy()

# remove 'none' in suggestion
df_output2['promotion_type_suggest'] = df_output2['promotion_type_suggest'].replace('none', np.nan)
# adjust: convert binary to boolean, then remove False as None
df_output2['approval'] = df_output2['approval'].replace({0.0: False, 1.0: True})
df_output2['approval'] = df_output2['approval'].apply(lambda x: None if x == False else x)
df_output2['promotion_type'] = df_output2['promotion_type_suggest'] * df_output2['approval']

# drop columns
df_output2 = df_output2.drop(columns=['cost', 'approval', 'probs_bronze', 'probs_gold', 'probs_silver', 'promotion_type_suggest'])

In [20]:
# sanity check
#df_output2.head()
df_output2[df_output2['promotion_type'].notna()].head()

,ltv,conv_ind,profit,promotion_type
8,2213.363453,1,1713.363453,bronze
15,1986.264107,1,1486.264107,bronze
20,3135.470329,1,2635.470329,bronze
21,1796.056726,1,1296.056726,bronze
27,1892.674417,1,1392.674417,bronze


In [21]:
# export to csv
df_output2.to_csv('submission_2_optimized.csv', index=False)